# ReAct Agent with Query Engine Tools

## Build Query Engine Tools

In [3]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
    load_index_from_storage,
)
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata

In [4]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0613")
# llm = OpenAI(temperature=0, model="gpt-4-0613")
service_context = ServiceContext.from_defaults(llm=llm)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3)
)
gpt4_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4-0613", temperature=0.3))

In [5]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/march")
    march_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/june")
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/sept")
    sept_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [6]:
if not index_loaded:
    # load data
    march_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()

    # build index
    march_index = VectorStoreIndex.from_documents(march_docs, service_context=service_context)
    june_index = VectorStoreIndex.from_documents(june_docs, service_context=service_context)
    sept_index = VectorStoreIndex.from_documents(sept_docs, service_context=service_context)

    # persist index
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")

In [7]:
march_engine = march_index.as_query_engine(similarity_top_k=3, service_context=service_context)
june_engine = june_index.as_query_engine(similarity_top_k=3, service_context=service_context)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, service_context=service_context)

In [8]:
from llama_index.tools.query_engine import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=f"Provides information about Uber quarterly financials ending September 2022",
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=f"Provides information about Uber quarterly financials ending June 2022",
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=f"Provides information about Uber quarterly financials ending March 2022",
)

query_engine_tools = [query_tool_march, query_tool_june, query_tool_sept]

## Setup Base ReAct Agent (gpt-3.5-turbo)

Here we define the baseline ReAct agent over our data, on top of gpt-3.5-turbo.

We run some example queries, and show that the ReAct agent can sometimes enter the incorrect reasoning loop to answer the question.

In [37]:
from llama_index.agent import ReActAgent
from llama_index.llms import OpenAI

In [38]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
# llm = OpenAI(model="gpt-4-0613")
base_agent = ReActAgent.from_tools(
    query_engine_tools, 
    llm=llm,
    verbose=True
)

In [ ]:
# gpt-3.5 generally gives the right response here 
response = base_agent.chat("Analyze Uber revenue growth over the last few quarters")
print(str(response))

In [ ]:
print(str(response))

In [97]:
# gpt-3.5 doesn't give the right response - it doesn't first search for the quarter with the highest revenue growth
response = base_agent.chat("Can you tell me about the risk factors in the quarter with the highest revenue growth?")
print(str(response))

Thought: To answer this question, I need to find the quarter with the highest revenue growth first. I will use the tools march_2022, june_2022, and sept_2022 to get the revenue data for each quarter.
Action: march_2022
Action Input: {'input': 'revenue'}
Observation: We generate substantially all of our revenue from fees paid by Drivers and Merchants for use of our platform. We act as an agent in these transactions by connecting consumers to Drivers and Merchants to facilitate a Trip, meal, or grocery delivery service. In certain markets, where we are responsible for mobility services, we present revenue from end-users on a gross basis. Our revenue is net of Driver and Merchant earnings and Driver incentives. We recognize revenue when a trip is complete.
Thought: The response from the tool doesn't provide a specific revenue figure for the March 2022 quarter. I'll need to ask for the revenue figure specifically.
Action: march_2022
Action Input: {'input': 'revenue figure'}
Observation: Th

## Generate Training/Eval Questions

Generate a synthetic dataset of questions to ask. To do this, we generate an initial set of questions over a "base" document (the March 2022 10Q), and then we use an LLM to generate variations of that question that can apply across multiple quarters. This allows us to more deeply stress-test the LLM reasoning capabilities.


In [12]:
from llama_index.evaluation import DatasetGenerator

In [105]:
base_question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from the Uber March 10Q filing, formulate "
    "a single question that captures an important fact from the context. "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    march_docs,
    question_gen_query=base_question_gen_query,
    service_context=gpt_35_context,
)

In [106]:
questions = dataset_generator.generate_questions_from_nodes(num=20)

In [107]:
questions

["What is the address of Uber Technologies, Inc.'s principal executive offices?",
 "What are the financial statements included in Uber's March 10Q filing?",
 'What are some of the factors that Uber identifies as potential impacts on its business operations and financial performance?',
 "What is the company's stance on updating forward-looking statements in their Quarterly Report on Form 10-Q?",
 "What is the total amount of cash and cash equivalents as of March 31, 2022, according to Uber's March 10Q filing?",
 'What was the net loss attributable to Uber Technologies, Inc. for the three months ended March 31, 2022?',
 'What was the comprehensive income (loss) attributable to Uber Technologies, Inc. for the three months ended March 31, 2022?',
 'What was the balance of non-redeemable non-controlling interests as of March 31, 2021, according to the Uber March 10Q filing?',
 'What was the net income (loss) for Uber Technologies, Inc. for the period ending March 31, 2022?',
 'What was the 

In [13]:
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate


vary_question_tmpl = """\
You are a financial assistant. Given a question over a 2023 Uber 10Q filing, your goal
is to generate up to {num_vary} variations of that question that might span multiple 10Q's.

This can include compare/contrasting different 10Qs, replacing the current quarter with
another quarter, or generating questions that can only be answered over multiple quarters (be creative!)

You are given a valid set of 10Q filings. Please only generate question variations that can be
answered in that set.

For example:
Base Question: What was the free cash flow of Uber in March 2023?
Valid 10Qs: [March 2023, June 2023, September 2023]
Question Variations:
What was the free cash flow of Uber in June 2023?
Can you compare/contrast the free cash flow of Uber in June/September 2023 and offer explanations for the change?
Did the free cash flow of Uber increase of decrease in 2023?

Now let's give it a shot! 

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""

def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[March 2022, June 2022, September 2022]"
    
    llm = OpenAI(model="gpt-4")
    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(prompt_tmpl.format(num_vary=num_vary, base_question=question, valid_10qs=VALID_10Q_STR))
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f'[{idx}] Original Question: {question}')
        print(f'[{idx}] Generated Question Variations: {cur_new_questions}')
        new_questions.extend(cur_new_questions)

    return new_questions

def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [120]:
new_questions = gen_question_variations(questions)

Original Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Generated Question Variations: ["Has the address of Uber Technologies, Inc.'s principal executive offices changed between March and September 2022?", "What was the address of Uber Technologies, Inc.'s principal executive offices in June 2022?", "Can you track the changes in the address of Uber Technologies, Inc.'s principal executive offices throughout 2022?"]
Original Question: What are the financial statements included in Uber's March 10Q filing?
Generated Question Variations: ["What are the financial statements included in Uber's June 2022 10Q filing?", "Can you compare and contrast the financial statements included in Uber's March and June 2022 10Q filings?", "How have the financial statements included in Uber's 10Q filings evolved over the course of 2022?"]
Original Question: What are some of the factors that Uber identifies as potential impacts on its business operations and financial

In [ ]:
len(new_questions)

In [127]:
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [131]:
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [15]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

## Use GPT-4 to Log Input/Output Pairs

We run the train questions through a GPT-4 powered ReAct agent to collect prompt outputs.

Every prompt call to the LLM is logged as an input/output pair. Since the ReAct loop can call the LLM multiple times, this means that multiple input/output pairs may be logged per user query.

Our `OpenAIFineTuningHandler` automatically collects prompt input/outputs when agent queries are run. This dataset can then be saved, in a dataset format `.jsonl` that you can directly feed to the OpenAI Finetuning endpoints.

In [16]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager
from llama_index.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [17]:
llm = OpenAI(model="gpt-4-0613")
gpt4_agent = ReActAgent.from_tools(
    query_engine_tools, 
    llm=llm,
    callback_manager=callback_manager,
    verbose=True
)

In [18]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = gpt4_agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

[0] Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'principal executive offices address'}
Observation: Sorry, but I can't help with that request.
Thought: The first tool didn't provide the needed information. I should try another tool.
Action: june_2022
Action Input: {'input': 'principal executive offices address'}
Observation: Sorry, but I can't help with that request.
Thought: The second tool also didn't provide the needed information. I should try the last tool.
Action: sept_2022
Action Input: {'input': 'principal executive offices address'}
Observation: Sorry, but I can't help with that request.
Response: I'm sorry, but I don't have the specific information about the address of Uber Technologies, Inc.'s principal executive offices. I recommend checking the official Uber website or their latest SEC filings for the most accurate information

In [22]:
# save events
finetuning_handler.save_finetuning_events("finetuning_events_10q.jsonl")

Wrote 184 examples to finetuning_events_10q.jsonl


In [23]:
# # TMP: REMOVE

# llm = OpenAI(model="gpt-4-0613")
# gpt4_agent = ReActAgent.from_tools(
#     query_engine_tools, 
#     llm=llm,
#     callback_manager=callback_manager,
#     verbose=True
# )

In [25]:
# # TMP RESPONSE
# gpt4_agent.query("What are the factors that have had an adverse impact on Uber's business and operations, as mentioned in the March 10Q filing?")

Thought: I need to use the march_2022 tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'adverse impact'}
Observation: The adverse impact mentioned in the context refers to the negative consequences or effects that certain events or circumstances can have on Uber's business, financial condition, and operating results. These adverse impacts can include factors such as economic, social, weather, and regulatory conditions, as well as the COVID-19 pandemic, security breaches, data privacy issues, cyberattacks, climate change risks, legal and regulatory risks, and market volatility. These adverse impacts can harm Uber's reputation, disrupt its operations, result in financial liabilities, and affect its ability to attract and retain users and personnel.
Response: The factors that have had an adverse impact on Uber's business and operations, as mentioned in the March 10Q filing, include economic, social, weather, and regulatory conditions. The COVID-19 pandemic h

Response(response="The factors that have had an adverse impact on Uber's business and operations, as mentioned in the March 10Q filing, include economic, social, weather, and regulatory conditions. The COVID-19 pandemic has also had a significant impact. Other factors include security breaches, data privacy issues, cyberattacks, climate change risks, legal and regulatory risks, and market volatility. These factors can harm Uber's reputation, disrupt its operations, result in financial liabilities, and affect its ability to attract and retain users and personnel.", source_nodes=[], metadata=None)

## Create `OpenAIFinetuneEngine`

We create an `OpenAIFinetuneEngine`: the finetune engine will launch a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

In [26]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events_10q.jsonl",
    # start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
)

In [27]:
finetune_engine.finetune()

Num examples: 184
First example:
{'role': 'system', 'content': '\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {\'title\': \'DefaultToolFnSchema\', \'description\': \'Default tool function Schema.\', \'type\': \'object\', \'properties\': {\'input\': {\'title\': \'Input\', \'type\': \'string\'}}, \'required\': [\'input\']}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {\'title\': \'DefaultToolF

In [30]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-OSUTIOyII1IwocEIB2ktcZhB at 0x2ba6868e0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-OSUTIOyII1IwocEIB2ktcZhB",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693700082,
  "finished_at": 1693700955,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:llamaindex::7uVHHzp7",
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [
    "file-rVuUfjj05GUQbWmnth2JT6W9"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-eUSkAcjIXOOSEtPRhSRR6qzb",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 449481
}

In [31]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

## Run Some Queries! (Compare Finetuned Agent vs. Base Agent)

We run some sample queries from the evaluation dataset over both our finetuned agent as well as the base agent.

We qualitatively look at their abilities to perform chain of thought prompting in order to arrive at the right answer.

**NOTE**: There's a big TODO to setup quantitative metrics so we can more rigorously evaluate the quality of any agent over an evaluation dataset! 

In [32]:
# Option 1: pass in ft_llm directly into ServiceContext
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
)

In [33]:
ft_agent = ReActAgent.from_tools(
    query_engine_tools, 
    llm=ft_llm,
    callback_manager=callback_manager,
    verbose=True
)

In [34]:
eval_questions = []
with open("eval_questions_10q.txt", "r") as f:
    for line in f:
        eval_questions.append(line.strip())

In [35]:
# try a sample question
qidx = 0
print(eval_questions[qidx])

What is the total fair value of Uber's financial assets as of March 31, 2022?


In [41]:
ft_agent._llm

OpenAI(callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x157bf5f60>, model='ft:gpt-3.5-turbo-0613:llamaindex::7uVHHzp7', temperature=0.3, max_tokens=None, additional_kwargs={}, max_retries=10, class_type='openai')

In [42]:
base_response = base_agent.query(eval_questions[qidx])
print(str(base_response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'financial_assets'}
Observation: The financial assets mentioned in the context include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. These assets are measured at fair value on a recurring basis and are categorized into three levels of the fair value hierarchy: Level 1, Level 2, and Level 3. Level 1 assets are valued based on quoted market prices, Level 2 assets are valued using readily available pricing sources or models with market observable inputs, and Level 3 assets are valued based on unobservable inputs and estimation techniques. The Level 3 assets primarily consist of non-marketable equity securities and the note receivable from a related party. The fair value of these assets is estimated using various valuation techniques, including the guideline public company approach and option-pricing models. The fair value of

In [43]:
ft_response = ft_agent.query(eval_questions[qidx])
print(str(ft_response))

Thought: I need to use the march_2022 tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'financial assets fair value'}
Observation: The financial assets of the company are measured at fair value on a recurring basis. These financial assets include non-marketable equity securities, marketable equity securities, and a note receivable from a related party. The fair value of these assets is determined based on the three-tier fair value hierarchy, with Level 1 valuations based on quoted market prices, Level 2 valuations obtained from readily available pricing sources or models using market observable inputs, and Level 3 valuations based on unobservable inputs and estimation techniques. The company did not make any transfers between the levels of the fair value hierarchy during the reporting period.
Thought: The tool didn't provide the specific fair value of Uber's financial assets as of March 31, 2022. I need to try again.
Action: march_2022
Action Input: {'inp

In [44]:
# try the original question that failed 
test_q = "Can you tell me about the risk factors in the quarter with the highest revenue growth?"
base_response = base_agent.query(test_q)
print(str(base_response))

Thought: I need to find the quarter with the highest revenue growth and then analyze the risk factors for that quarter.
Action: march_2022
Action Input: {'input': 'revenue_growth'}
Observation: The revenue growth for the company in the three months ended March 31, 2022, compared to the same period in 2021, was 136%. This increase in revenue was primarily driven by an increase in Gross Bookings of 35%, primarily due to increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19, and a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace. Additionally, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business model changes in the UK.
Thought: The quarter with the highest revenue growth is the quarter ending March 2022. Now I need to analyze the risk factors for that quarter.
Action: march_2022
Action Input: {'input': 'risk_factors'}
Observation: The company faces vari

In [45]:
# NOTE: this successfully looks at each quarter for revenue growth but still fa
ft_response = ft_agent.query(test_q)
print(str(ft_response))

Thought: I need to use the tools to get the revenue growth data for each quarter.
Action: march_2022
Action Input: {'input': 'revenue growth'}
Observation: The revenue for the company has increased significantly compared to the same period last year. The revenue growth is primarily driven by an increase in Gross Bookings, particularly in Mobility Trip volumes as the business recovers from the impacts of COVID-19. Additionally, the acquisition of Transplace has contributed to an increase in Freight revenue. The company has also seen an increase in Delivery revenue and Mobility revenue due to certain business model changes. Overall, the revenue growth for the company has been positive.
Thought: Now I have the revenue growth information for the first quarter. I need to compare it with the revenue growth of the second and third quarters to determine which quarter had the highest growth.
Action: june_2022
Action Input: {'input': 'revenue growth'}
Observation: The context information does no

**Observations**: The finetuned model does much better than the base model in terms of reasoning about the current sequence of steps. It passes more detailed answers to the downstream tools and is more capable of refining its approach when initial queries don't work. This applies even if the answer isn't actually found within the context (which is a function of our automatic dataset generation capabilities). 